Import Necessary Libraries

In [2263]:
import pandas as pd
import tempfile
import os
from datetime import date as d
from datetime import timedelta

Set Up API

In [2264]:
import statsapi as api

Set Up Beautiful Soup

In [2265]:
import re
import requests
from bs4 import BeautifulSoup, SoupStrainer

Set Up Selenium

In [2266]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

For killing background chrome processes:

In [2267]:
#import psutil
#import subprocess
#import signal

#def kill_chrome_processes():
#    for proc in psutil.process_iter():
#        if 'chrome' in proc.name() or 'chromedriver' in proc.name:
#            proc.kill()

Input Player Name and Date


NEED TO CHECK AND FIX ALL CELLS: IF STATS FOR SOMETHING DO NOT EXIST NEED TO DEAL WITH THAT!

ALSO NEED TO ENSURE CORRECT VALUES ARE PASSED IN FOR COMPUTING CORRECT VALUES

NEED TO CHECK PLAYER AVG OVER X GAMES - NEED TO COMPUTE PLAYER AVG OVER LAST GAME FOR PAST SEASONS AS WELL

NEED TO ENSURE THAT ALL DATA IS OF CORRECT TYPE, CORRECT VALUES, CORRECT YEARS, AND IF NONE NEEDS TO BE HANDLED

FIX CURRENT SET UP FOR COMPUTING AVERAGES. SHIT IS BUGGY AS FUCK

IF IT APPEARS TO BE GOOD: THERE WILL BE :P NEXT TO IT. IF NEEDS ADJUSTMENTS, VERIFICATION, OR REWORK, :( NEXT TO IT

ESPECIALLY DECIDE HOW TO DEAL WITH EDGE CASES AND DIVISION BY 0.

WALKS, HIT BY PITCH, AND SAC FLIES DO NOT COUNT AS AN AT BAT

TOTAL BASES ARE ONLY BASES REACHED VIA HITS.

In [2268]:
first_name = input("Enter Player First Name: ")
last_name = input('Enter Player Last Name: ')
#date = input('Enter Date: ')
date = '2024-07-24'
og_date = date
year = int(date[:4])
last_year = year - 1
print(year)
player_name = f'{first_name.lower()} {last_name.lower()}'


2024


fetch player_id :P

In [2269]:
player_info = api.lookup_player(player_name, season=year)
player_name = player_info[0]['fullName']
player_id = player_info[0]['id']
team_id = player_info[0]['currentTeam']['id']
team_info = api.lookup_team(team_id, season=year)
team_name = team_info[0]['name']

print(player_name)
print(player_id)
print(team_name)
print(team_id)

CJ Abrams
682928
Washington Nationals
120


Gather batter_hand, batter_age, , batter_strike_zone_top, batter_strike_zone_bottom :P

In [2270]:
person_info = api.get('people',{'personIds':player_id, 'season':year,'sportId':1})
batter_hand = person_info['people'][0]['batSide']['code']
batter_age = person_info['people'][0]['currentAge']
batter_strike_zone_top = person_info['people'][0]['strikeZoneTop']
batter_strike_zone_bottom = person_info['people'][0]['strikeZoneBottom']
batter_starting_year = int(person_info['people'][0]['mlbDebutDate'][:4])
batter_year = year - batter_starting_year


print(batter_age)
print(batter_year)
print(batter_hand)
print(batter_strike_zone_top)
print(batter_strike_zone_bottom)

23
2
L
3.52
1.63


fetch game_pk, home, opp_id, venue_id, sPitcher_id, sPitcher_hand :P

In [2271]:
game_info = api.schedule(date=date,team=team_id,sportId=1)
game_pk = game_info[0]['game_id']
home_team_id = game_info[0]['home_id']
if home_team_id == team_id:
    home = True
    opp_id = game_info[0]['away_id']
    sPitcher_name = game_info[0]['away_probable_pitcher']
else:
    home = False
    opp_id = game_info[0]['home_id']
    sPitcher_name = game_info[0]['home_probable_pitcher']

if sPitcher_name == '':
    sPitcher_name = None
    sPitcher_id = None
    sPitcher_hand = None
else:
    sPitcher_info = api.lookup_player(sPitcher_name)
    sPitcher_id = sPitcher_info[0]['id']
    sPitcher_info_2 = api.get('people',{'personIds':sPitcher_id, 'season':year,'sportId':1})
    sPitcher_hand = sPitcher_info_2['people'][0]['pitchHand']['code']

venue_id = game_info[0]['venue_id']
venue_name = game_info[0]['venue_name']

if game_info[0]['status'] == 'Final':
    game_status = True
else:
    game_status = False

opp_name = api.lookup_team(opp_id, season=year)[0]['name']

print(game_status)
print(game_pk)
print(opp_name)
print(opp_id)
print(sPitcher_name)
print(sPitcher_id)
print(sPitcher_hand)
print(venue_name)
print(venue_id)
print(home)

True
744824
San Diego Padres
135
Matt Waldron
663362
R
Nationals Park
3309
True


fetch player game stats: temp, wind_speed, hp_ump_id, game_time. Fetch list of proj_ump_ids if Home Plate Umpire not yet specified :(

What should happen when this data isnt uploaded yet?

In [2272]:
game_boxscore_info = api.boxscore_data(gamePk=game_pk)

temp = None
wind_speed = None
game_time = None

for x in game_boxscore_info['gameBoxInfo']:
    if x.get('label') == 'Weather':
        temp_info = x['value']
        temps = re.findall(r'\d+', temp_info)
        if temps:
            temp = int(temps[0])
            

    if x.get('label') == 'Wind':
        wind_speed_info = x['value']
        winds = re.findall(r'\d+', wind_speed_info)
        if winds:
            wind_speed = int(winds[0])
            

    if x.get('label') == 'First pitch':
        time_str = x['value'].strip('.')
        time_part, period = time_str.split(' ')
        hour_str, min_str = time_part.split(':')

        if period.lower() == 'am':
            game_time_str = f'{hour_str}{min_str}'
        
        elif period.lower() == 'pm':
            game_time_str = f'{int(hour_str) + 12}{min_str}'
        
        game_time = int(game_time_str)


    if x.get('label') == 'Umpires':
        ump_info = x['value']
        proj_ump_ids = []
        proj_umps = []
        if 'HP:' in ump_info:
            pattern = r'HP: (.*?)\.'
            match = re.search(pattern, ump_info)
            if match:
                hp_ump = match.group(1)
                hp_ump_info = api.get('jobs_umpires', {})
                for person in hp_ump_info['roster']:
                    if person['person'].get('fullName') == hp_ump:
                        hp_ump_id = person['person'].get('id')
                        proj_umps.append(hp_ump)
                        proj_ump_ids.append(hp_ump_id)

        elif ump_info:
            umps = re.findall(r'[\w\s]+', ump_info)
            umps = [ump.strip() for ump in umps if ump.strip]
            ump_list = list(umps) + [None]*(4-len(umps))
            umps_tuple = tuple(ump_list[:4])
            for ump in umps_tuple:
                ump_info = api.get('jobs_umpires',{})
                for person in ump_info['roster']:
                    if person['person'].get('fullName') == ump:
                        ump_id = person['person'].get('id')
                        proj_umps.append(ump)
                        proj_ump_ids.append(ump_id)

print(temp)
print(wind_speed)
print(game_time)
print(proj_ump_ids)
print(proj_umps)



84
3
1845
[511890]
['Quinn Wolcott']


fetch player game data: :P
- batting position

In [2273]:
game_content = api.boxscore_data(gamePk=game_pk)
if home == True:
    batters_info = game_content['homeBatters']
else:
    batters_info = game_content['awayBatters']

batting_position = None

for batter in batters_info:
    if batter.get('personId') == player_id:
        batting_position = int(batter.get('battingOrder')[0])

print (batting_position)

1


fetch game player stats :P
- If game_status == True
    - gather player stats
- If game_status == False
    - set player stats to None

In [2274]:
if game_status == True:
    game_stats_info = api.get('person_stats',{'personId':player_id,'gamePk':game_pk})
    game_stats = game_stats_info['stats'][0]['splits'][2]['stat']
    pa = game_stats.get('plateAppearances')
    ab = game_stats.get('atBats')
    h = game_stats.get('hits')
    r = game_stats.get('runs')
    rbi = game_stats.get('rbi')
    hr = game_stats.get('homeRuns')
    tb = game_stats.get('totalBases')
    k = game_stats.get('strikeOuts')
    bb = game_stats.get('baseOnBalls')

else:
    pa = None
    ab = None
    h = None
    r = None
    rbi = None
    hr = None
    tb = None
    k = None
    bb = None

batter_game_stats_dict = {'pa': pa, 'ab': ab, 'h': h, 'r': r, 'rbi': rbi, 'hr': hr, 'tb': tb, 'k': k, 'bb': bb}

batter_game_stats_df = pd.DataFrame([batter_game_stats_dict])

print(batter_game_stats_df)


   pa  ab  h  r  rbi  hr  tb  k  bb
0   3   3  0  0    0   0   0  0   0


Determine Whether Most Recent Streak Averages, or Historical Streak Averages are Needed for Player

Format Player URL for baseballreference.com

In [2275]:
initial = last_name[:1].lower()

full_name = f'{first_name} {last_name}'.lower()

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                formatted_player_key = player_link[11:20]

print(formatted_player_key)

abramcj01


Collect Data for Games Played Up to Desired Date
    - For entirely current year, or entirely last year

Format Player Game Data URL

In [2276]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

if date == season_start_date:
    date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'
else:
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={year}#batting_gamelogs'

Set Correct Date

In [2277]:
day = int(date[8:10])
month = int(date[5:7])

date_dt = d(year, month, day)

new_date_dt = date_dt - timedelta(days=1)

date = new_date_dt.strftime("%Y-%m-%d")

day = int(date[8:10])
month = int(date[5:7])

print(date)
print(day)
print(month)

2024-07-23
23
7


Scrape Player Game Data

In [2278]:
response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    df = pd.DataFrame(data)

    df.rename(columns={
        'batting_avg': 'BA',
        'onbase_perc': 'OBP',
        'slugging_perc': 'SLG',
    }, inplace=True)

    all_columns = df.columns.tolist()

    columns_to_keep = ['game_num', 'date_game', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

    columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

    correct_columns_df = df.drop(columns_to_drop, axis=1)

    columns_to_convert_int = ['game_num', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

    columns_to_convert_float = ['BA', 'OBP', 'SLG']

    for column in columns_to_convert_int:
         correct_columns_df[column] = correct_columns_df[column].astype(int)

    for column in columns_to_convert_float:
        correct_columns_df[column] = correct_columns_df[column].astype(float)

    correct_columns_df['TB'] = correct_columns_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

    column_order = ['game_num', 'date_game', 'PA', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
    correct_columns_df = correct_columns_df[column_order]

print(correct_columns_df)


    game_num   date_game  PA  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA  \
0          1  2024-03-28   4   4  0  0    0   0   0   0   0   0   2  0.000   
1          2  2024-03-30   5   2  1  3    0   0   1   0   3   3   1  0.167   
2          3  2024-03-31   5   4  1  1    1   0   0   1   4   0   1  0.200   
3          4  2024-04-01   5   5  2  0    1   0   0   0   2   0   1  0.267   
4          5  2024-04-03   4   4  2  0    2   0   0   0   2   0   1  0.316   
5          6  2024-04-04   5   5  2  1    1   0   0   1   5   0   2  0.333   
6          7  2024-04-05   4   4  1  0    0   0   0   0   1   0   0  0.321   
7          8  2024-04-09   4   4  2  1    3   0   0   1   5   0   1  0.344   
8          9  2024-04-10   4   4  0  0    0   0   0   0   0   0   0  0.306   
9         10  2024-04-12   5   5  0  0    0   0   0   0   0   0   3  0.268   
10        11  2024-04-13   5   4  2  0    1   1   1   0   5   1   1  0.289   
11        12  2024-04-14   5   4  1  1    0   1   0   0   2   1 

Season Averages Up to Desired Date

In [2279]:
season_stats = {}

game_data = correct_columns_df.iloc[-1]

games_played = game_data['game_num']

print(games_played)

season_stats['pa_per_game'] = correct_columns_df['PA'].mean()
season_stats['ab_per_game'] = correct_columns_df['AB'].mean()
season_stats['h_per_game'] = correct_columns_df['H'].mean()
season_stats['r_per_game'] = correct_columns_df['R'].mean()
season_stats['rbi_per_game'] = correct_columns_df['RBI'].mean()
season_stats['hr_per_game'] = correct_columns_df['HR'].mean()
season_stats['tb_per_game'] = correct_columns_df['TB'].mean()
season_stats['bb_per_game'] = correct_columns_df['BB'].mean()
season_stats['k_per_game'] = correct_columns_df['SO'].mean()
season_stats['ba_szn'] = game_data['BA']
season_stats['obp_szn'] = game_data['OBP']
season_stats['slg_szn'] = game_data['SLG']

season_stats_df = pd.DataFrame([season_stats])

print(season_stats_df)

93
   pa_per_game  ab_per_game  h_per_game  r_per_game  rbi_per_game  \
0     4.462366     3.989247    1.053763    0.645161      0.526882   

   hr_per_game  tb_per_game  bb_per_game  k_per_game  ba_szn  obp_szn  slg_szn  
0      0.16129     1.892473     0.322581    0.870968   0.264    0.336    0.474  


Streak Average Data Collection

In [2280]:
if games_played < 30:
    last_season_end_date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']

    season_end_day = int(last_season_end_date[8:10])
    season_end_month = int(last_season_end_date[5:7])

    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'

    response = requests.get(search_url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

    if table_rows:

        data = []

        for row in table_rows:

            row_data = {}

            th = row.find('th')
            
            row_num = th.text
            
            row_data['game_num'] = row_num

            for td in row.find_all('td'):

                stat_type = td['data-stat']

                if stat_type == 'date_game':
                    stat_value = td['csk'][:10]
                else:
                    stat_value = td.text

                row_data[stat_type] = stat_value

            if row_data:

                game_day = int(row_data['date_game'][8:10])
                game_month = int(row_data['date_game'][5:7])

                if game_month < season_end_month:
                        data.append(row_data)
                elif game_month == season_end_month:
                    if game_day <= season_end_day:
                        data.append(row_data)

        df = pd.DataFrame(data)

        print(df)

        df.rename(columns={
            'batting_avg': 'BA',
            'onbase_perc': 'OBP',
            'slugging_perc': 'SLG',
        }, inplace=True)

        all_columns = df.columns.tolist()

        columns_to_keep = ['game_num', 'date_game', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

        columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

        last_season_df = df.drop(columns_to_drop, axis=1)

        columns_to_convert_int = ['game_num', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

        columns_to_convert_float = ['BA', 'OBP', 'SLG']

        for column in columns_to_convert_int:
            last_season_df[column] = last_season_df[column].astype(int)

        for column in columns_to_convert_float:
            last_season_df[column] = last_season_df[column].astype(float)

        last_season_df['TB'] = last_season_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

        column_order = ['game_num', 'date_game', 'PA', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
        last_season_df = last_season_df[column_order]

    print(last_season_df)




Last 7 Averages:

In [2281]:
if games_played < 7:
    carry_over_games = 7 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_7_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_7_game_num = games_played - 7
    last_7_df = correct_columns_df.iloc[last_7_game_num:]

print(last_7_df)

last_7_stats = {}

last_7_stats['pa_last_7'] = last_7_df['PA'].mean()
last_7_stats['ab_last_7'] = last_7_df['AB'].mean()
last_7_stats['h_last_7'] = last_7_df['H'].mean()
last_7_stats['r_last_7'] = last_7_df['R'].mean()
last_7_stats['rbi_last_7'] = last_7_df['RBI'].mean()
last_7_stats['hr_last_7'] = last_7_df['HR'].mean()
last_7_stats['tb_last_7'] = last_7_df['TB'].mean()
last_7_stats['bb_last_7'] = last_7_df['BB'].mean()
last_7_stats['k_last_7'] = last_7_df['SO'].mean()
last_7_stats['ba_last_7'] = round(last_7_df['H'].sum() / last_7_df['AB'].sum(), 3)
last_7_stats['obp_last_7'] = round((last_7_df['H'].sum() + last_7_df['BB'].sum() + last_7_df['HBP'].sum()) / last_7_df['AB'].sum(), 3)
last_7_stats['slg_last_7'] = round( last_7_df['TB'].sum() / last_7_df['AB'].sum(), 3)

last_7_avg_df = pd.DataFrame([last_7_stats])

print(last_7_avg_df)

    game_num   date_game  PA  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA  \
86        87  2024-07-12   5   5  1  1    0   0   0   0   1   0   2  0.270   
87        88  2024-07-13   5   5  1  1    2   0   0   1   4   0   1  0.269   
88        89  2024-07-14   4   4  1  1    0   0   1   0   3   0   1  0.268   
89        90  2024-07-19   4   4  0  0    0   0   0   0   0   0   0  0.265   
90        91  2024-07-20   5   5  2  1    1   0   0   0   2   0   0  0.267   
91        92  2024-07-21   4   4  1  0    0   0   0   0   1   0   0  0.267   
92        93  2024-07-23   4   4  0  0    0   0   0   0   0   0   1  0.264   

      OBP    SLG  HBP  
86  0.345  0.481    0  
87  0.344  0.486    0  
88  0.343  0.489    0  
89  0.339  0.483    0  
90  0.340  0.482    0  
91  0.339  0.480    0  
92  0.336  0.474    0  
   pa_last_7  ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  \
0   4.428571   4.428571  0.857143  0.571429    0.428571   0.142857   1.571429   

   bb_last_7  k_last

Last 15 Averages

In [2282]:
if games_played < 15:
    carry_over_games = 15 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_15_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_15_game_num = games_played - 15
    last_15_df = correct_columns_df.iloc[last_15_game_num:]

print(last_15_df)

last_15_stats = {}

last_15_stats['pa_last_15'] = last_15_df['PA'].mean()
last_15_stats['ab_last_15'] = last_15_df['AB'].mean()
last_15_stats['h_last_15'] = last_15_df['H'].mean()
last_15_stats['r_last_15'] = last_15_df['R'].mean()
last_15_stats['rbi_last_15'] = last_15_df['RBI'].mean()
last_15_stats['hr_last_15'] = last_15_df['HR'].mean()
last_15_stats['tb_last_15'] = last_15_df['TB'].mean()
last_15_stats['bb_last_15'] = last_15_df['BB'].mean()
last_15_stats['k_last_15'] = last_15_df['SO'].mean()
last_15_stats['ba_last_15'] = round(last_15_df['H'].sum() / last_15_df['AB'].sum(), 3)
last_15_stats['obp_last_15'] = round((last_15_df['H'].sum() + last_15_df['BB'].sum() + last_15_df['HBP'].sum()) / last_15_df['AB'].sum(), 3)
last_15_stats['slg_last_15'] = round( last_15_df['TB'].sum() / last_15_df['AB'].sum(), 3)

last_15_avg_df = pd.DataFrame([last_15_stats])

print(last_15_avg_df)

    game_num   date_game  PA  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA  \
78        79  2024-07-04   4   4  0  0    0   0   0   0   0   0   1  0.279   
79        80  2024-07-05   6   4  1  0    2   1   0   0   2   2   1  0.279   
80        81  2024-07-06   5   3  1  3    1   0   0   1   4   1   1  0.280   
81        82  2024-07-07   5   4  2  0    0   0   0   0   2   0   2  0.282   
82        83  2024-07-08   4   4  0  0    0   0   0   0   0   0   0  0.279   
83        84  2024-07-09   5   3  0  0    0   0   0   0   0   0   1  0.276   
84        85  2024-07-10   4   3  0  0    0   0   0   0   0   1   1  0.274   
85        86  2024-07-11   5   4  0  0    0   0   0   0   0   0   2  0.271   
86        87  2024-07-12   5   5  1  1    0   0   0   0   1   0   2  0.270   
87        88  2024-07-13   5   5  1  1    2   0   0   1   4   0   1  0.269   
88        89  2024-07-14   4   4  1  1    0   0   1   0   3   0   1  0.268   
89        90  2024-07-19   4   4  0  0    0   0   0   0   0   0 

Last 30 Averages

In [2283]:
if games_played < 30:
    carry_over_games = 30 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_30_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_30_game_num = games_played - 30
    last_30_df = correct_columns_df.iloc[last_30_game_num:]

print(last_30_df)

last_30_stats = {}

last_30_stats['pa_last_30'] = last_30_df['PA'].mean()
last_30_stats['ab_last_30'] = last_30_df['AB'].mean()
last_30_stats['h_last_30'] = last_30_df['H'].mean()
last_30_stats['r_last_30'] = last_30_df['R'].mean()
last_30_stats['rbi_last_30'] = last_30_df['RBI'].mean()
last_30_stats['hr_last_30'] = last_30_df['HR'].mean()
last_30_stats['tb_last_30'] = last_30_df['TB'].mean()
last_30_stats['bb_last_30'] = last_30_df['BB'].mean()
last_30_stats['k_last_30'] = last_30_df['SO'].mean()
last_30_stats['ba_last_30'] = round(last_30_df['H'].sum() / last_30_df['AB'].sum(), 3)
last_30_stats['obp_last_30'] = round((last_30_df['H'].sum() + last_30_df['BB'].sum() + last_30_df['HBP'].sum()) / last_30_df['AB'].sum(), 3)
last_30_stats['slg_last_30'] = round( last_30_df['TB'].sum() / last_30_df['AB'].sum(), 3)

last_30_avg_df = pd.DataFrame([last_30_stats])

print(last_30_avg_df)

    game_num   date_game  PA  AB  H  R  RBI  2B  3B  HR  TB  BB  SO     BA  \
63        64  2024-06-15   4   4  1  1    0   1   0   0   2   0   0  0.257   
64        65  2024-06-16   4   3  1  0    0   0   0   0   1   1   0  0.258   
65        66  2024-06-18   4   4  2  0    0   0   0   0   2   0   0  0.261   
66        67  2024-06-21   6   5  3  2    1   2   0   0   5   1   1  0.267   
67        68  2024-06-22   5   5  3  2    2   1   0   1   7   0   1  0.273   
68        69  2024-06-23   4   4  0  1    0   0   0   0   0   0   2  0.270   
69        70  2024-06-24   5   5  3  2    0   1   0   0   4   0   0  0.275   
70        71  2024-06-25   5   3  3  1    1   0   0   0   3   2   0  0.283   
71        72  2024-06-26   3   3  0  0    0   0   0   0   0   0   1  0.280   
72        73  2024-06-28   4   2  0  0    0   0   0   0   0   0   0  0.278   
73        74  2024-06-29   4   3  2  2    2   1   0   1   6   1   0  0.282   
74        75  2024-06-30   4   2  1  0    0   0   0   0   1   2 

Merge Streak Data

In [2284]:
streak_data_df = pd.concat([last_7_avg_df, last_15_avg_df, last_30_avg_df], axis=1)
print(streak_data_df)

   pa_last_7  ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  \
0   4.428571   4.428571  0.857143  0.571429    0.428571   0.142857   1.571429   

   bb_last_7  k_last_7  ba_last_7  obp_last_7  slg_last_7  pa_last_15  \
0        0.0  0.714286      0.194       0.194       0.355         4.6   

   ab_last_15  h_last_15  r_last_15  rbi_last_15  hr_last_15  tb_last_15  \
0         4.0   0.666667   0.466667          0.4    0.133333    1.266667   

   bb_last_15  k_last_15  ba_last_15  obp_last_15  slg_last_15  pa_last_30  \
0    0.266667   0.933333       0.167        0.317        0.317         4.5   

   ab_last_30  h_last_30  r_last_30  rbi_last_30  hr_last_30  tb_last_30  \
0         3.8   1.066667        0.7     0.433333    0.133333    1.766667   

   bb_last_30  k_last_30  ba_last_30  obp_last_30  slg_last_30  
0    0.466667        0.8       0.281        0.465        0.465  


Merge Season Averages and Streak Data



In [2285]:
batter_season_stats_df = pd.concat([streak_data_df, season_stats_df], axis=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(batter_season_stats_df)

   pa_last_7  ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  \
0   4.428571   4.428571  0.857143  0.571429    0.428571   0.142857   1.571429   

   bb_last_7  k_last_7  ba_last_7  obp_last_7  slg_last_7  pa_last_15  \
0        0.0  0.714286      0.194       0.194       0.355         4.6   

   ab_last_15  h_last_15  r_last_15  rbi_last_15  hr_last_15  tb_last_15  \
0         4.0   0.666667   0.466667          0.4    0.133333    1.266667   

   bb_last_15  k_last_15  ba_last_15  obp_last_15  slg_last_15  pa_last_30  \
0    0.266667   0.933333       0.167        0.317        0.317         4.5   

   ab_last_30  h_last_30  r_last_30  rbi_last_30  hr_last_30  tb_last_30  \
0         3.8   1.066667        0.7     0.433333    0.133333    1.766667   

   bb_last_30  k_last_30  ba_last_30  obp_last_30  slg_last_30  pa_per_game  \
0    0.466667        0.8       0.281        0.465        0.465     4.462366   

   ab_per_game  h_per_game  r_per_game  rbi_per_game  hr_per_game  \

Fetch Starting Pitcher Season Data


MASSIVE:

PLAN FOR FUTURE FOR PITCHER, BVP, BVOPP, OPP, BVB Data

DESIGN THE 'Starting Pitcher Efficiency Index'

Step 1: SP Data
    - Gather Data such as:
        - ip per game
        - batters faced per game
        - plate appearances per game
        - pitches per game
        - at bats per game

Step 2: Include these new stat categories:
    - Batter Vs Opponent - Previous Batter stats from each game against the opponent this season
    - Team Vs Opponent Stats - Avg Team Stats for Each Final Boxscore of Previous Season Matchups
    - Pitcher vs Team Stats - Avg Pitcher Stats Vs Batter's team from previous mathcups
    - Batter Vs Pitcher - Batter Vs Pitcher Stats from IP or PA from each game featuring the two
    - Batter Vs Bullpen (Batter Vs Team - Batter Vs Pitcher)

    Goal is to determine how valuable the batter's plate appearences with the starting pitcher compared to the batter's plate appearances with the relievers and closers.
    What percentage of hits come from the starting pitcher vs the relievers. Want to try to predict the number of plate appearnaces the batter will get during a game with this starting pitcher, and then use this value to predict hits by comparing the likelihood the batter will get a hit within the plate appearances against the starting pitcher and the likelihood the batter will get a hit within the plate appearances against the relievers and closers. Then using these averages, predict how likely the batter is to get a hit in this game. Basically instead of just using starting pitcher, want to treat both the starting pitcher and other pitchers as an entire game to predict hits. This will be done by leveraging the innings pitched by the starter and other pitchers with the stats of starter v batter and bullpen v batter

Stats to Collect:
- sp_season_stats -- starting pitcher season averages up to the desired date. x
- bv_sp_stats -- direct batter vs starting pitcher stats measured by per plate appearance. x
- sp_v_team -- starting pitcher game stats in games vs team this season.
- bv_opp_stats -- batter game stats in games vs opp this season. x
- opp_stats -- opposing team per game stats 
- bv_opp_sp_stats -- batter game stats in games vs opp where the starting pitcher started this season. x
- team_v_opp_stats -- overall team stats in games vs opp this season.

Format Pitcher URL

In [2286]:
sp_name_parts = sPitcher_name.split()

sp_first_name = sp_name_parts[0].lower()
sp_last_name = sp_name_parts[1].lower()

initial = sp_last_name[:1]

full_name = f'{sp_first_name} {sp_last_name}'

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                #formatted_pitcher_key = player_link[11:20]

pattern = r'/players/[a-z]/([a-z]{1,5}[a-z]{2}\d{2})\.shtml'

match = re.search(pattern, player_link)

if match:
    formatted_pitcher_key = match.group(1)

print(formatted_pitcher_key)

waldrma01


In [2287]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_pitcher_key}&t=p&year={year}#pitching_gamelogs'

response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('pitching_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    sp_df = pd.DataFrame(data)

    sp_df.rename(columns={
            'earned_run_avg': 'ERA',
            'batters_faced': 'BF',
            'pitches': 'P',
            'strikes_total': 'S',
        }, inplace=True)

    sp_all_columns = sp_df.columns.tolist()

    sp_columns_to_keep = ['game_num', 'date_game', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'BF', 'P', 'S', 'AB', '2B', '3B']

    sp_columns_to_drop = [column for column in sp_all_columns if column not in sp_columns_to_keep]

    sp_column_df = sp_df.drop(sp_columns_to_drop, axis=1)

    sp_columns_to_convert_int = ['game_num', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'BF', 'P', 'S', 'AB', '2B', '3B']

    sp_columns_to_convert_float = ['IP', 'ERA']

    for column in sp_columns_to_convert_int:
        sp_column_df[column] = sp_column_df[column].astype(int)

    for column in sp_columns_to_convert_float:
        sp_column_df[column] = sp_column_df[column].astype(float)
    
    #print(sp_column_df)

    sp_column_df['WHIP'] = sp_column_df.apply(lambda row: ((row['H'] + row['BB']) / row['IP']) if row['IP'] > 0.0 else (row['H'] + row['BB']), axis=1)

    sp_column_df['TB'] = sp_column_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

    sp_column_df['IP'] = sp_column_df['IP'].apply(lambda ip: int(ip) + (1/3) if round(ip % 1, 1) == 0.1 else int(ip) + (2/3) if round(ip % 1, 1) == 0.2 else ip)

    sp_column_df['TO'] = sp_column_df.apply(lambda row: row['IP']*3, axis=1)

    sp_column_df['S%'] = sp_column_df.apply(lambda row: row['S'] / row['P'], axis=1)

    sp_column_df['BAA'] = sp_column_df.apply(lambda row: row['H'] / row['AB'], axis=1)

    sp_column_order = ['game_num', 'date_game', 'IP', 'P', 'S', 'S%', 'BF', 'AB', 'H', '2B', '3B', 'R', 'ER', 'HR', 'TB', 'BB', 'SO', 'TO', 'HBP', 'ERA', 'WHIP', 'BAA']
                       
    sp_complete_df = sp_column_df[sp_column_order]

    print(sp_complete_df)

    game_num   date_game        IP    P   S        S%  BF  AB  H  2B  3B  R  \
0          1  2024-04-01  4.000000   91  61  0.670330  22  21  9   2   0  4   
1          2  2024-04-07  5.333333   85  53  0.623529  21  20  3   1   0  1   
2          3  2024-04-13  5.000000   93  53  0.569892  23  19  3   0   0  1   
3          4  2024-04-19  4.666667   96  58  0.604167  22  21  7   1   0  5   
4          5  2024-04-24  6.000000   91  58  0.637363  25  22  4   0   0  1   
5          6  2024-04-29  6.000000   97  63  0.649485  24  22  6   4   0  4   
6          7  2024-05-05  3.000000   65  40  0.615385  19  16  8   3   0  8   
7          8  2024-05-11  5.333333   82  51  0.621951  20  18  2   1   0  2   
8          9  2024-05-17  5.666667   98  62  0.632653  23  21  5   1   0  1   
9         10  2024-05-23  5.000000   90  58  0.644444  22  19  6   1   1  2   
10        11  2024-05-28  7.000000   85  67  0.788235  27  27  6   1   0  0   
11        12  2024-06-03  6.333333   95  55  0.57894

Pitcher Season Stats Up to Desired Date

In [2288]:
sp_season_stats = {}

sp_game_data = sp_complete_df.iloc[-1]

sp_games_started = sp_game_data['game_num']

print(sp_games_started)

sp_season_stats['sp_games_started'] = sp_games_started
sp_season_stats['sp_ip_per_game'] = sp_complete_df['IP'].mean()
sp_season_stats['sp_p_per_game'] = sp_complete_df['P'].mean()
sp_season_stats['sp_s_per_game'] = sp_complete_df['S'].mean()
sp_season_stats['sp_s%'] = sp_complete_df['S'].sum() / sp_complete_df['P'].sum()
sp_season_stats['sp_bf_per_game'] = sp_complete_df['BF'].mean()
sp_season_stats['sp_ab_per_game'] = sp_complete_df['AB'].mean()
sp_season_stats['sp_h_per_game'] = sp_complete_df['H'].mean()
sp_season_stats['sp_r_per_game'] = sp_complete_df['R'].mean()
sp_season_stats['sp_er_per_game'] = sp_complete_df['ER'].mean()
sp_season_stats['sp_hr_per_game'] = sp_complete_df['HR'].mean()
sp_season_stats['sp_tb_per_game'] = sp_complete_df['TB'].mean()
sp_season_stats['sp_bb_per_game'] = sp_complete_df['BB'].mean()
sp_season_stats['sp_k_per_game'] = sp_complete_df['SO'].mean()
sp_season_stats['sp_k%'] = sp_complete_df['SO'].sum() / sp_complete_df['BF'].sum()
sp_season_stats['sp_to_per_game'] = sp_complete_df['TO'].mean()
sp_season_stats['sp_era'] = sp_game_data['ERA']
sp_season_stats['sp_whip_per_game'] = sp_complete_df['WHIP'].mean()
sp_season_stats['sp_whip_szn'] = ((sp_complete_df['H'].sum() + sp_complete_df['BB'].sum() ) / sp_complete_df['IP'].sum())
sp_season_stats['sp_baa'] = sp_complete_df['H'].sum() / sp_complete_df['AB'].sum()

sp_season_stats_df = pd.DataFrame([sp_season_stats])

print(sp_season_stats_df)

20
   sp_games_started  sp_ip_per_game  sp_p_per_game  sp_s_per_game     sp_s%  \
0                20        5.633333           91.4           58.3  0.637856   

   sp_bf_per_game  sp_ab_per_game  sp_h_per_game  sp_r_per_game  \
0            23.3            21.3            5.0            2.4   

   sp_er_per_game  sp_hr_per_game  sp_tb_per_game  sp_bb_per_game  \
0            2.25             0.6             8.1            1.55   

   sp_k_per_game     sp_k%  sp_to_per_game  sp_era  sp_whip_per_game  \
0           4.95  0.212446            16.9    3.59          1.296308   

   sp_whip_szn    sp_baa  
0     1.162722  0.234742  


Format Batter vs Starting Pitcher URL :P

In [2289]:
formatted_batter_name = player_name.replace(' ', '%20')

team_schedule = api.schedule(start_date=season_start_date, end_date=date, team=team_id, opponent=opp_id)

if len(team_schedule) > 0:

    games_vs_opp = []
    games_vs_sp = []
    games_vs_sp_pks = []

    for game in team_schedule:
        if game['game_type'] != 'S' and game['status'] != 'Postponed':
            for log in api.get('person_stats', {'personId': player_id, 'gamePk': game['game_id']})['stats'][0]['splits']:
                if log.get('group') == 'hitting' and log['stat']:
                    games_vs_opp.append(game['game_id'])
                    if game['home_id'] == opp_id:
                        if game['home_probable_pitcher'] == sPitcher_name:
                            games_vs_sp.append(game['game_date'])
                            games_vs_sp_pks.append(game['game_id'])
                    elif game['away_id'] == opp_id:
                        if game['away_probable_pitcher'] == sPitcher_name:
                            games_vs_sp.append(game['game_date'])
                            games_vs_sp_pks.append(game['game_id'])
    
    num_games_vs_sp = len(games_vs_sp)
    num_games_vs_opp = len(games_vs_opp)

    game_strings = [f'%22{game_vs_opp}%22' for game_vs_opp in games_vs_opp]

    comma_game_strings =f'[' + ','.join(game_strings) + ']'

    if num_games_vs_sp == 0:
        bvp_url = None
    else:
        bvp_url = f'https://baseballsavant.mlb.com/illustrator?playerId={player_id}&playerType=batter&name={formatted_batter_name}&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22{sPitcher_id}%22]&balls=[]&battedBallType=[]&years=[{year}]&strikes=[]&outs=[]&selectedGames=' + comma_game_strings + '&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7'


else:
    num_games_vs_sp = 0
    num_games_vs_opp = 0
    games_vs_opp = None
    games_vs_sp = None
    bvp_url = None

print(num_games_vs_sp)
print(num_games_vs_opp)

print(bvp_url)
print(games_vs_opp)
print(games_vs_sp)
print(games_vs_sp_pks)


1
4
https://baseballsavant.mlb.com/illustrator?playerId=682928&playerType=batter&name=CJ%20Abrams&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22663362%22]&balls=[]&battedBallType=[]&years=[2024]&strikes=[]&outs=[]&selectedGames=[%22745397%22,%22745399%22,%22745400%22,%22744828%22]&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7
[745397, 745399, 745400, 744828]
['2024-06-24']
[745397]


Gather b_vs_opp data

In [2290]:
bv_opp_data_dict = {}

if num_games_vs_opp > 0:

    bv_opp_raw_data = []

    for game in games_vs_opp:
        game_vs_opp_log = api.get('person_stats', {'personId': player_id, 'gamePk': game})['stats'][0]['splits']
        game_vs_opp_data = {}
        for group in game_vs_opp_log:
            if group['group'] == 'hitting':
                for s in group['stat']:
                    game_vs_opp_data[s] = group['stat'][s]
        bv_opp_raw_data.append(game_vs_opp_data)
    
    bv_opp_raw_data_df = pd.DataFrame(bv_opp_raw_data)

    bv_opp_raw_data_df.rename(columns={
        'runs': 'bv_opp_r',
        'doubles': '2B',
        'triples': '3B',
        'homeRuns': 'bv_opp_hr',
        'strikeOuts': 'bv_opp_k',
        'baseOnBalls': 'bv_opp_bb',
        'hits': 'bv_opp_h',
        'hitByPitch': 'HBP',
        'atBats': 'bv_opp_ab',
        'plateAppearances': 'bv_opp_pa',
        'totalBases': 'bv_opp_tb',
        'rbi': 'bv_opp_rbi',
        'sacFlies': 'SF'
    }, inplace=True)

    bv_opp_columns = bv_opp_raw_data_df.columns.to_list()

    bv_opp_columns_to_keep = ['bv_opp_pa', 'bv_opp_ab', 'bv_opp_h', 'bv_opp_r', 'bv_opp_rbi', '2B', '3B', 'bv_opp_hr', 'bv_opp_tb', 'bv_opp_bb', 'bv_opp_k', 'HBP', 'SF']

    bv_opp_columns_to_drop = [column for column in bv_opp_columns if column not in bv_opp_columns_to_keep]

    bv_opp_columns_df = bv_opp_raw_data_df.drop(bv_opp_columns_to_drop, axis=1)

    bv_opp_columns_df.astype(int)

    for column in bv_opp_columns_df.columns.to_list():
        bv_opp_data_dict[column] = bv_opp_columns_df[column].mean()

    bv_opp_data_dict['bv_opp_ba'] = round(bv_opp_data_dict['bv_opp_h'] / bv_opp_data_dict['bv_opp_ab'], 3)
    bv_opp_data_dict['bv_opp_obp'] = round((bv_opp_data_dict['bv_opp_h'] + bv_opp_data_dict['bv_opp_bb'] + bv_opp_data_dict['HBP']) / (
        bv_opp_data_dict['bv_opp_ab'] + bv_opp_data_dict['bv_opp_bb'] + bv_opp_data_dict['HBP'] + bv_opp_data_dict['SF']), 3)
    bv_opp_data_dict['bv_opp_slg'] = round(bv_opp_data_dict['bv_opp_tb'] / bv_opp_data_dict['bv_opp_ab'], 3)

    for key in ['2B', '3B', 'HBP', 'SF']:
        bv_opp_data_dict.pop(key)
    
    bv_opp_data_dict['bv_opp_games_played'] = num_games_vs_opp

else:
    bv_opp_data_dict = {'bv_opp_games_played': None, 'bv_opp_pa': None, 'bv_opp_ab': None, 'bv_opp_h': None,'bv_opp_r': None, 'bv_opp_rbi': None, 'bv_opp_hr': None, 'bv_opp_tb': None, 'bv_opp_bb': None,
                         'bv_opp_k': None, 'bv_opp_ba': None, 'bv_opp_obp': None, 'bv_opp_slg': None}

bv_opp_data_df = pd.DataFrame([bv_opp_data_dict])

bv_opp_column_order = ['bv_opp_games_played', 'bv_opp_pa', 'bv_opp_ab', 'bv_opp_h', 'bv_opp_r', 'bv_opp_rbi', 'bv_opp_hr', 'bv_opp_tb', 'bv_opp_bb', 'bv_opp_k', 'bv_opp_ba', 'bv_opp_obp', 'bv_opp_slg']

bv_opp_df = bv_opp_data_df[bv_opp_column_order]

print(bv_opp_df)

   bv_opp_games_played  bv_opp_pa  bv_opp_ab  bv_opp_h  bv_opp_r  bv_opp_rbi  \
0                    4       4.25       3.75       1.5      0.75        0.25   

   bv_opp_hr  bv_opp_tb  bv_opp_bb  bv_opp_k  bv_opp_ba  bv_opp_obp  \
0        0.0       1.75        0.5       0.5        0.4       0.471   

   bv_opp_slg  
0       0.467  


Gather bv_opp_sp_stats

In [2291]:
bv_opp_sp_data_dict = {}

if num_games_vs_sp > 0:

    bv_opp_sp_raw_data = []

    for game in games_vs_sp_pks:
        game_vs_opp_sp_log = api.get('person_stats', {'personId': player_id, 'gamePk': game})['stats'][0]['splits']
        game_vs_opp_sp_data = {}
        for group in game_vs_opp_sp_log:
            if group['group'] == 'hitting':
                for s in group['stat']:
                    game_vs_opp_sp_data[s] = group['stat'][s]
        bv_opp_sp_raw_data.append(game_vs_opp_sp_data)
    
    bv_opp_sp_raw_data_df = pd.DataFrame(bv_opp_sp_raw_data)

    bv_opp_sp_raw_data_df.rename(columns={
        'runs': 'bv_opp_sp_r',
        'doubles': '2B',
        'triples': '3B',
        'homeRuns': 'bv_opp_sp_hr',
        'strikeOuts': 'bv_opp_sp_k',
        'baseOnBalls': 'bv_opp_sp_bb',
        'hits': 'bv_opp_sp_h',
        'hitByPitch': 'HBP',
        'atBats': 'bv_opp_sp_ab',
        'plateAppearances': 'bv_opp_sp_pa',
        'totalBases': 'bv_opp_sp_tb',
        'rbi': 'bv_opp_sp_rbi',
        'sacFlies': 'SF'
    }, inplace=True)

    bv_opp_sp_columns = bv_opp_sp_raw_data_df.columns.to_list()

    bv_opp_sp_columns_to_keep = ['bv_opp_sp_pa', 'bv_opp_sp_ab', 'bv_opp_sp_h', 'bv_opp_sp_r', 'bv_opp_sp_rbi', '2B', '3B', 
                                 'bv_opp_sp_hr', 'bv_opp_sp_tb', 'bv_opp_sp_bb', 'bv_opp_sp_k', 'HBP', 'SF']

    bv_opp_sp_columns_to_drop = [column for column in bv_opp_sp_columns if column not in bv_opp_sp_columns_to_keep]

    bv_opp_sp_columns_df = bv_opp_sp_raw_data_df.drop(bv_opp_sp_columns_to_drop, axis=1)

    bv_opp_sp_columns_df.astype(int)

    for column in bv_opp_sp_columns_df.columns.to_list():
        bv_opp_sp_data_dict[column] = bv_opp_sp_columns_df[column].mean()
    
    bv_opp_sp_data_dict['bv_opp_sp_ba'] = round(bv_opp_sp_data_dict['bv_opp_sp_h'] / bv_opp_sp_data_dict['bv_opp_sp_ab'], 3)
    bv_opp_sp_data_dict['bv_opp_sp_obp'] = round((bv_opp_sp_data_dict['bv_opp_sp_h'] + bv_opp_sp_data_dict['bv_opp_sp_bb'] + bv_opp_sp_data_dict['HBP']) / (
        bv_opp_sp_data_dict['bv_opp_sp_ab'] + bv_opp_sp_data_dict['bv_opp_sp_bb'] + bv_opp_sp_data_dict['HBP'] + bv_opp_sp_data_dict['SF']), 3)
    bv_opp_sp_data_dict['bv_opp_sp_slg'] = round(bv_opp_sp_data_dict['bv_opp_sp_tb'] / bv_opp_sp_data_dict['bv_opp_sp_ab'], 3)

    for key in ['2B', '3B', 'HBP', 'SF']:
        bv_opp_sp_data_dict.pop(key)

else:
    bv_opp_sp_data_dict = {'bv_opp_sp_pa': None, 'bv_opp_sp_ab': None, 'bv_opp_sp_h': None,'bv_opp_sp_r': None, 'bv_opp_sp_rbi': None, 'bv_opp_sp_hr': None, 'bv_opp_sp_tb': None, 
                           'bv_opp_sp_bb': None, 'bv_opp_sp_k': None, 'bv_opp_sp_ba': None, 'bv_opp_sp_obp': None, 'bv_opp_sp_slg': None}

bv_opp_sp_stats_df = pd.DataFrame([bv_opp_sp_data_dict])

bv_opp_sp_column_order = ['bv_opp_sp_pa', 'bv_opp_sp_ab', 'bv_opp_sp_h', 'bv_opp_sp_r', 'bv_opp_sp_rbi', 'bv_opp_sp_hr', 'bv_opp_sp_tb', 
                          'bv_opp_sp_bb', 'bv_opp_sp_k', 'bv_opp_sp_ba', 'bv_opp_sp_obp', 'bv_opp_sp_slg']

bv_opp_sp_df = bv_opp_sp_stats_df[bv_opp_sp_column_order]

print(bv_opp_sp_df)

   bv_opp_sp_pa  bv_opp_sp_ab  bv_opp_sp_h  bv_opp_sp_r  bv_opp_sp_rbi  \
0           5.0           5.0          3.0          2.0            0.0   

   bv_opp_sp_hr  bv_opp_sp_tb  bv_opp_sp_bb  bv_opp_sp_k  bv_opp_sp_ba  \
0           0.0           4.0           0.0          0.0           0.6   

   bv_opp_sp_obp  bv_opp_sp_slg  
0            0.6            0.8  


Gather sp_vs_team stats

Gather opp_stats

Gather team_v_opp_stats

INITIALIZING CHROMEDRIVER

In [2292]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--disable-images")
prefs = {
    'profile.default_content_setting_values': {
        'images': 2,
        'javascript': 2,
        'stylesheet': 2
    }
}
options.add_experimental_option('prefs', prefs)

#service = Service(chrome_driver_path)
#driver = webdriver.Chrome(service=service, options=options)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

Gather Raw BVP Data

In [2293]:
if bvp_url is not None:
    bvp_raw_data = {}

    #service = Service(chrome_driver_path)
    #driver = webdriver.Chrome(service=service, options=options)
    
    #with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    try:
        driver.get(bvp_url)

        driver.implicitly_wait(10)

        stats_boxes = driver.find_elements(By.CLASS_NAME, 'stats-box')

        for stats_box in stats_boxes:
            stat_type = stats_box.find_element(By.CLASS_NAME, 'stats-title')
            stat_value = stats_box.find_element(By.CLASS_NAME, 'stats-value')
            bvp_raw_data[stat_type.text] = stat_value.text
        
        for key in bvp_raw_data.keys():
            bvp_raw_data[key] = float(bvp_raw_data[key])
    
    except Exception as e:
        bvp_raw_data = None

    #finally:
        #driver.close()
        #driver.quit()
        #kill_chrome_processes()

else:
    bvp_raw_data = None

print(bvp_raw_data)

{'Pitches': 10.0, 'PA': 3.0, 'H': 2.0, '1B': 1.0, '2B': 1.0, '3B': 0.0, 'HR': 0.0, 'AVG': 0.667, 'SO': 0.0, 'wOBA': 0.713, 'K%': 0.0, 'EV': 96.7, 'LA': 22.0}


Manipulate bvp data

TO DO: Remove normalization to 9 innings and keep as total per game.
Consider: switching to per plate appearance
Consider: instead of just looking at bvp, look at entire game where that pitcher was sp
            - record stats per plate appearance against starting pitcher
            - record stats per plate appearance against entire bullpen
            - most importantly, determine avg of plate appearances against sp per game and stats associated with those plate appearances
            - big idea: want to determine how many plate appearances the batter will get vs this sp, and then determine the likelihood of player getting a hit within those plate appearances.
            - Then want to determine how many plate appearances the batter will get after sp, then then determine the likelihood of a player getting a hit within those plate appearances.
            - Combine both of these stats to predict the amount of plate appearances the batter will get per game, and how likely they are to get a hit given the split of those plate appearances 

In [2294]:
bvp_data = {'bvp_games_faced': None,
            'bvp_ip_total': None,
            'bvp_ip_per_game': None,
            'bvp_p_total': None,
            'bvp_p_per_game': None,
            'bvp_pa_total': None,
            'bvp_pa_per_game': None,
            'bvp_h_total': None,
            'bvp_h_per_game': None,
            'bvp_hr_total': None,
            'bvp_hr_per_game': None,
            'bvp_tb_total': None,
            'bvp_tb_per_game': None,
            'bvp_k_total': None,
            'bvp_k_per_game': None,
            'bvp_k%': None,
            'bvp_woba': None,
            'bvp_ba': None,
            }

if bvp_raw_data is not None:
    bvp_total_data = {}
    sp_ip = 0.0

    for d in games_vs_sp:
        sp_ip += float(sp_column_df[sp_column_df['date_game'] == d]['IP'].iloc[0])

    sp_ip_per_bvp = round(sp_ip / num_games_vs_sp, 3)
    bvp_total_data['bvp_ip_total'] = sp_ip
    bvp_total_data['bvp_ip_per_game'] = sp_ip_per_bvp

    bvp_total_data['bvp_p_total'] = bvp_raw_data['Pitches']
    bvp_total_data['bvp_p_per_game'] = round(bvp_raw_data['Pitches'] / num_games_vs_sp, 3)
    bvp_total_data['bvp_pa_total'] = bvp_raw_data['PA']
    bvp_total_data['bvp_pa_per_game'] = round(bvp_raw_data['PA'] / num_games_vs_sp, 3)
    bvp_total_data['bvp_h_total'] = bvp_raw_data['H']
    bvp_total_data['bvp_h_per_game'] = round(bvp_raw_data['H'] / num_games_vs_sp, 3)
    bvp_total_data['bvp_1B_adj'] = bvp_raw_data['1B']
    bvp_total_data['bvp_2B_adj'] = bvp_raw_data['2B']
    bvp_total_data['bvp_3B_adj'] = bvp_raw_data['3B']
    bvp_total_data['bvp_hr_total'] = bvp_raw_data['HR']
    bvp_total_data['bvp_hr_per_game'] = round(bvp_raw_data['HR'] / num_games_vs_sp, 3)
    bvp_total_data['bvp_k_total'] = bvp_raw_data['SO']
    bvp_total_data['bvp_k_per_game'] = round(bvp_raw_data['SO'] / num_games_vs_sp, 3)
    
    bvp_total_data['bvp_tb_total'] = (bvp_total_data['bvp_1B_adj'] + 2 * bvp_total_data['bvp_2B_adj']
                                                  + 3 * bvp_total_data['bvp_3B_adj'] + 4 * bvp_total_data['bvp_hr_per_game'])
    bvp_total_data['bvp_tb_per_game'] = round(bvp_total_data['bvp_tb_total'] / num_games_vs_sp, 3)

    bvp_data['bvp_ba'] = bvp_raw_data['AVG']
    bvp_data['bvp_k%'] = bvp_raw_data['K%']
    bvp_data['bvp_woba'] = bvp_raw_data['wOBA']

    for i in ['bvp_1B_adj', 'bvp_2B_adj', 'bvp_3B_adj']:
        bvp_total_data.pop(i)
    
    for total in bvp_total_data.keys():
        bvp_data[total] = bvp_total_data[total]
    
    bvp_data['bvp_games_faced'] = num_games_vs_sp
    
    bvp_data_df = pd.DataFrame([bvp_data]).astype(float)

else:
    bvp_data_df = pd.DataFrame([bvp_data])

print(bvp_data_df)
        

   bvp_games_faced  bvp_ip_total  bvp_ip_per_game  bvp_p_total  \
0              1.0           6.0              6.0         10.0   

   bvp_p_per_game  bvp_pa_total  bvp_pa_per_game  bvp_h_total  bvp_h_per_game  \
0            10.0           3.0              3.0          2.0             2.0   

   bvp_hr_total  bvp_hr_per_game  bvp_tb_total  bvp_tb_per_game  bvp_k_total  \
0           0.0              0.0           3.0              3.0          0.0   

   bvp_k_per_game  bvp_k%  bvp_woba  bvp_ba  
0             0.0     0.0     0.713   0.667  


Fetch Umpire Stats
- If Home Plate Ump announced
    - Return data for only HP ump 
- If Home Plate Ump not yet announced
    - Fetch data for all 4 umps
    - hp_ump stats = avg of 4 umps stats

In [2295]:
if len(proj_ump_ids) > 0:

    #service = Service(chrome_driver_path)
    #driver = webdriver.Chrome(service=service, options=options)

    #with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    try:
        driver.get('https://swishanalytics.com/mlb/mlb-umpire-factors')

        table = driver.find_element(By.CSS_SELECTOR, "div.table-responsive.table-umpires > table")

        rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")

        umpire_data = []

        for row in rows:
            umpire_name = row.find_element(By.CSS_SELECTOR, "td.umps-name").text.strip()
            if umpire_name in proj_umps:
                row_data = [cell.text.strip() for cell in row.find_elements(By.CSS_SELECTOR, "td")]
                umpire_data.append(row_data[3:])
        
        for u in umpire_data:
            i = 0
            while i < len(u):
                if u[i].endswith('%'):
                    u[i] = float(u[i].strip('%')) / 100
                elif u[i].endswith('x'):
                    u[i] = float(u[i].strip('x'))
                else:
                    u[i] = float(u[i])
                i += 1
    
    except Exception as e:
        umpire_data = [[None for _ in range(12)]]
                
    #finally:
        #driver.close()
        #driver.quit()

else:
    umpire_data = [[None for _ in range(12)]]


print(umpire_data)


[[0.217, 0.081, 9.04, 0.255, 0.322, 0.416, 0.92, 0.88, 0.98, 1.05, 1.0, 1.0]]


In [2296]:
umpire_data_df = pd.DataFrame(columns=['ump_k%', 'ump_bb%', 'ump_r_per_game', 'ump_ba', 'ump_obp', 'ump_slg', 
                                       'ump_k_boost', 'ump_bb_boost', 'ump_r_boost', 'ump_ba_boost', 'ump_obp_boost', 'ump_slg_boost'])

if len(umpire_data) > 1:
    avg_list = [sum(x) / len(umpire_data) for x in zip(*umpire_data)]
    
    umpire_data_df.loc[0] = avg_list

else:
    umpire_data_df.loc[0] = umpire_data[0]

print(umpire_data_df)

    

   ump_k%  ump_bb%  ump_r_per_game  ump_ba  ump_obp  ump_slg  ump_k_boost  \
0   0.217    0.081            9.04   0.255    0.322    0.416         0.92   

   ump_bb_boost  ump_r_boost  ump_ba_boost  ump_obp_boost  ump_slg_boost  
0          0.88         0.98          1.05            1.0            1.0  


Fetch Venue Stats:

*FUTURE CHANGES: DETERMINE A BETTER WAY TO SET DATA FOR SWITCH HITTERS.

In [2297]:
#options = Options()
#options.add_argument("--headless")
#options.add_argument("--disable-gpu")
#options.add_argument("--no-sandbox")
#options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--disable-extensions")
#options.add_argument("--disable-images")
#prefs = {
#    'profile.default_content_setting_values': {
#        'images': 2,
#        'javascript': 2,
#        'stylesheet': 2
#    }
#}
#options.add_experimental_option('prefs', prefs)
#
#service = Service(chrome_driver_path)
#driver = webdriver.Chrome(service=service, options=options)
#
#with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:

try:
    driver.get('https://swishanalytics.com/mlb/mlb-park-factors')

    venue_element = WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.XPATH, f"//h4[contains(text(), '{venue_name}')]")))

    venue_table = venue_element.find_element(By.XPATH, "./following::table[1]")

    venue_rows = venue_table.find_elements(By.TAG_NAME, 'tr')

    venue_data = []

    for row in venue_rows[1:]:
        venue_cells = row.find_elements(By.TAG_NAME, 'td')
        venue_row_data = [cell.text for cell in venue_cells]

        if batter_hand == 'L':
            venue_data.append(float(venue_row_data[0]))
        elif batter_hand == 'R':
            venue_data.append(float(venue_row_data[2]))
        else:
            venue_data.append(round((float(venue_row_data[0]) + float(venue_row_data[2])) / 2, 3))

except Exception as e:
    venue_data = [None for _ in range(9)]

#finally:
    #None
    #driver.close()
    #driver.quit()

print(venue_data)

[1.09, 0.83, 1.04, 1.0, 1.01, 1.0, 1.03, 1.01, 1.05]


In [2298]:
venue_data_df = pd.DataFrame(columns=['ven_hr_x', 'ven_3B_x', 'ven_2B_x', 'ven_1B_x', 'ven_ba_x', 'ven_obp_x', 
                                      'ven_slg_x', 'ven_woba_x', 'ven_r_x'])

venue_data_df.loc[0] = venue_data

print(venue_data_df)

   ven_hr_x  ven_3B_x  ven_2B_x  ven_1B_x  ven_ba_x  ven_obp_x  ven_slg_x  \
0      1.09      0.83      1.04       1.0      1.01        1.0       1.03   

   ven_woba_x  ven_r_x  
0        1.01     1.05  


QUIT CHROME DRIVER AFTER SCRAPING COMPLETES

In [2299]:
driver.close()
driver.quit()

Combine General Data into DataFrame

In [2300]:
batter_general_data = {
    'player_first_name': first_name,
    'player_last_name': last_name,
    'player_id': player_id,
    'game_pk': game_pk,
    'game_status': game_status,
    'game_num': games_played + 1,
    'date': og_date,
    'game_time': game_time,
    'temp': temp,
    'wind_speed': wind_speed,
    'venue_name': venue_name,
    'venue_id': venue_id,
    'team_name': team_name,
    'team_id': team_id,
    'opp_name': opp_name,
    'opp_id': opp_id,
    'home': home,
    'batter_hand': batter_hand,
    'batting_pos': batting_position,
    'batter_age': batter_age,
    'batter_year': batter_year,
    'batter_k_zone_top': batter_strike_zone_top,
    'batter_k_zone_bottom': batter_strike_zone_bottom,
    's_pitcher_first_name': sp_first_name,
    's_pitcher_last_name': sp_last_name,
    's_pitcher_id': sPitcher_id,
    's_pitcher_hand': sPitcher_hand,
}

if len(proj_ump_ids) < 1:
    batter_general_data['ump_id'] = None
    batter_general_data['ump_name'] = None

else:
    batter_general_data['ump_id'] = proj_ump_ids[0]
    batter_general_data['ump_name'] = proj_umps[0]

batter_general_data_df = pd.DataFrame([batter_general_data])

print(batter_general_data_df)

  player_first_name player_last_name  player_id  game_pk  game_status  \
0                cj           abrams     682928   744824         True   

   game_num        date  game_time  temp  wind_speed      venue_name  \
0        94  2024-07-24       1845    84           3  Nationals Park   

   venue_id             team_name  team_id          opp_name  opp_id  home  \
0      3309  Washington Nationals      120  San Diego Padres     135  True   

  batter_hand  batting_pos  batter_age  batter_year  batter_k_zone_top  \
0           L            1          23            2               3.52   

   batter_k_zone_bottom s_pitcher_first_name s_pitcher_last_name  \
0                  1.63                 matt             waldron   

   s_pitcher_id s_pitcher_hand  ump_id       ump_name  
0        663362              R  511890  Quinn Wolcott  


Merge Batter General Data and Batter Season Data

In [2301]:
batter_stats_df = pd.concat([batter_general_data_df, batter_game_stats_df, batter_season_stats_df], axis=1)

print(batter_stats_df)

  player_first_name player_last_name  player_id  game_pk  game_status  \
0                cj           abrams     682928   744824         True   

   game_num        date  game_time  temp  wind_speed      venue_name  \
0        94  2024-07-24       1845    84           3  Nationals Park   

   venue_id             team_name  team_id          opp_name  opp_id  home  \
0      3309  Washington Nationals      120  San Diego Padres     135  True   

  batter_hand  batting_pos  batter_age  batter_year  batter_k_zone_top  \
0           L            1          23            2               3.52   

   batter_k_zone_bottom s_pitcher_first_name s_pitcher_last_name  \
0                  1.63                 matt             waldron   

   s_pitcher_id s_pitcher_hand  ump_id       ump_name  pa  ab  h  r  rbi  hr  \
0        663362              R  511890  Quinn Wolcott   3   3  0  0    0   0   

   tb  k  bb  pa_last_7  ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  \
0   0  0   0   4.428571

Merge bv Stats

In [2302]:
bv_stats_df = pd.concat([bvp_data_df, bv_opp_sp_df, bv_opp_df], axis=1)

print(bv_stats_df)

   bvp_games_faced  bvp_ip_total  bvp_ip_per_game  bvp_p_total  \
0              1.0           6.0              6.0         10.0   

   bvp_p_per_game  bvp_pa_total  bvp_pa_per_game  bvp_h_total  bvp_h_per_game  \
0            10.0           3.0              3.0          2.0             2.0   

   bvp_hr_total  bvp_hr_per_game  bvp_tb_total  bvp_tb_per_game  bvp_k_total  \
0           0.0              0.0           3.0              3.0          0.0   

   bvp_k_per_game  bvp_k%  bvp_woba  bvp_ba  bv_opp_sp_pa  bv_opp_sp_ab  \
0             0.0     0.0     0.713   0.667           5.0           5.0   

   bv_opp_sp_h  bv_opp_sp_r  bv_opp_sp_rbi  bv_opp_sp_hr  bv_opp_sp_tb  \
0          3.0          2.0            0.0           0.0           4.0   

   bv_opp_sp_bb  bv_opp_sp_k  bv_opp_sp_ba  bv_opp_sp_obp  bv_opp_sp_slg  \
0           0.0          0.0           0.6            0.6            0.8   

   bv_opp_games_played  bv_opp_pa  bv_opp_ab  bv_opp_h  bv_opp_r  bv_opp_rbi  \
0         

Merge Pitching Stats

In [2303]:
#pitching_stats_df = pd.concat([sp_season_stats_df, sp_vs_team_stats_df], axis=1)
pitching_stats_df = pd.concat([sp_season_stats_df], axis=1)

print(pitching_stats_df)

   sp_games_started  sp_ip_per_game  sp_p_per_game  sp_s_per_game     sp_s%  \
0                20        5.633333           91.4           58.3  0.637856   

   sp_bf_per_game  sp_ab_per_game  sp_h_per_game  sp_r_per_game  \
0            23.3            21.3            5.0            2.4   

   sp_er_per_game  sp_hr_per_game  sp_tb_per_game  sp_bb_per_game  \
0            2.25             0.6             8.1            1.55   

   sp_k_per_game     sp_k%  sp_to_per_game  sp_era  sp_whip_per_game  \
0           4.95  0.212446            16.9    3.59          1.296308   

   sp_whip_szn    sp_baa  
0     1.162722  0.234742  


Merge Team Stats

In [2304]:
#team_stats_df = pd.concat([opp_stats_df, team_v_opp_stats_df], axis=1)

#print(team_stats_df)

Merge Umpire and Venue Stats

In [2305]:
factor_stats_df = pd.concat([umpire_data_df, venue_data_df], axis=1)

print(factor_stats_df)

   ump_k%  ump_bb%  ump_r_per_game  ump_ba  ump_obp  ump_slg  ump_k_boost  \
0   0.217    0.081            9.04   0.255    0.322    0.416         0.92   

   ump_bb_boost  ump_r_boost  ump_ba_boost  ump_obp_boost  ump_slg_boost  \
0          0.88         0.98          1.05            1.0            1.0   

   ven_hr_x  ven_3B_x  ven_2B_x  ven_1B_x  ven_ba_x  ven_obp_x  ven_slg_x  \
0      1.09      0.83      1.04       1.0      1.01        1.0       1.03   

   ven_woba_x  ven_r_x  
0        1.01     1.05  


FORM COMPLETE RAW DATA DF

In [2306]:
#complete_raw_data_df = pd.concat([batter_stats_df, bv_stats_df, pitching_stats_df, team_stats_df, factor_stats_df], axis=1)
complete_raw_data_df = pd.concat([batter_stats_df, bv_stats_df, pitching_stats_df, factor_stats_df], axis=1)

print(complete_raw_data_df)

##Need to determine a way to have the game time be the time the game is scheduled so it can show up pre-game
#ADD A GAME IN SERIES NUMBER 

  player_first_name player_last_name  player_id  game_pk  game_status  \
0                cj           abrams     682928   744824         True   

   game_num        date  game_time  temp  wind_speed      venue_name  \
0        94  2024-07-24       1845    84           3  Nationals Park   

   venue_id             team_name  team_id          opp_name  opp_id  home  \
0      3309  Washington Nationals      120  San Diego Padres     135  True   

  batter_hand  batting_pos  batter_age  batter_year  batter_k_zone_top  \
0           L            1          23            2               3.52   

   batter_k_zone_bottom s_pitcher_first_name s_pitcher_last_name  \
0                  1.63                 matt             waldron   

   s_pitcher_id s_pitcher_hand  ump_id       ump_name  pa  ab  h  r  rbi  hr  \
0        663362              R  511890  Quinn Wolcott   3   3  0  0    0   0   

   tb  k  bb  pa_last_7  ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  \
0   0  0   0   4.428571